# **Sistema de Recomendacao Baseado em Conteudo - **











In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

np.set_printoptions(suppress=True)

Leitura do arquivo de Transacoes 

Leitura do arquivo de clientes

In [3]:
df_t = pd.read_csv('transacoes_1.4.csv')

**Leitura primeiras linhas do Arquivo de transacao**

In [4]:
df_t.head()

,carrinho,produto,descricao,categoria
0,380962,4059805069114,CAMISETA SUPERNOVA M RAWSTE M,RUNNING
1,380962,4055017073826,MEIA LINER CUSHION 3S 3P WHITE/WHITE/BLACK 3538,TRAINING
2,380962,4058032608264,CAMISETA MC FREELIFT PRIME GRNNIT M,TRAINING
3,380962,4059323533418,TENIS UltraBOOST All Terrain GRETHR/GRETWO/NOB...,RUNNING
4,329797,4057288560432,BERMUDA KNIT MGREYH/TRAGRE XL,ATHLETICS


In [5]:
df_t.shape

(1960186, 4)

In [6]:
df_t.duplicated(subset='descricao').sum() # Verificando duplicidades Descricao

1885091

In [7]:
df_t2 = df_t.drop_duplicates(subset='descricao')

In [8]:
df_t2.duplicated(subset='descricao').sum()

0

In [9]:
sample_size = 15000
df_t2 = df_t2.sample(n=sample_size, replace=False, random_state=490)

df_t2 = df_t2.reset_index()
df_t2 = df_t2.drop('index',axis=1)

In [10]:
df_t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   carrinho   15000 non-null  int64 
 1   produto    15000 non-null  int64 
 2   descricao  14999 non-null  object
 3   categoria  14999 non-null  object
dtypes: int64(2), object(2)
memory usage: 468.9+ KB


In [11]:
# combine all strings:
df_t3 = df_t2.drop(['carrinho','produto'],axis=1)

In [12]:
df_t3.head()

,descricao,categoria
0,CAMISETA FL30 X GF GRA M BLACK/GRESIX XL,TRAINING
1,TENIS ZX 5K BOOST W HAZROS/AMBLUS/SONINK 35,ORIGINALS
2,TENIS SEELEY FTWWHT/ENEBLU/ENERGY 43,SKATEBOARDING
3,TENIS MARATHON 5923 BLUE/SILVMT/CROYAL 41,ORIGINALS
4,TENIS 3MC VULC CBLACK/FTWWHT/GLOPNK 34,SKATEBOARDING


Combinando e criando uma unica string

In [13]:
df_t3['data'] = (df_t3['descricao'] + ' ' + df_t3['categoria']).astype(str)

print(df_t3['data'].head())

0    CAMISETA FL30 X GF GRA M BLACK/GRESIX XL TRAINING
1    TENIS ZX 5K BOOST W HAZROS/AMBLUS/SONINK 35 OR...
2    TENIS SEELEY FTWWHT/ENEBLU/ENERGY 43 SKATEBOAR...
3    TENIS MARATHON 5923 BLUE/SILVMT/CROYAL 41 ORIG...
4    TENIS 3MC VULC CBLACK/FTWWHT/GLOPNK 34 SKATEBO...
Name: data, dtype: object


In [15]:
vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df_t3['data'])

Do Scikit Learn vamos fazer os vetores

The variable “vectorized” is a sparse matrix with a numeric representation of the strings we extracted.

In [16]:
similarities = cosine_similarity(vectorized)

In [17]:
print(similarities)

[[1.         0.         0.         ... 0.         0.         0.14433757]
 [0.         1.         0.12598816 ... 0.         0.08908708 0.        ]
 [0.         0.12598816 1.         ... 0.         0.10101525 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.08908708 0.10101525 ... 0.         1.         0.        ]
 [0.14433757 0.         0.         ... 0.         0.         1.        ]]


In [18]:
df_t3 = pd.DataFrame(similarities, columns=df_t3['descricao'], index=df_t3['descricao']).reset_index()

df_t3.head()

descricao,descricao,CAMISETA FL30 X GF GRA M BLACK/GRESIX XL,TENIS ZX 5K BOOST W HAZROS/AMBLUS/SONINK 35,TENIS SEELEY FTWWHT/ENEBLU/ENERGY 43,TENIS MARATHON 5923 BLUE/SILVMT/CROYAL 41,TENIS 3MC VULC CBLACK/FTWWHT/GLOPNK 34,CONJUNTO W TS HD TGHT LEGINK/LEGINK/BLUE S,BERMUDA WAFFLE WONWHI L,MALA FAVOURITES BLACK/BLACK/WHITE NT,CAMISETA FLSPR A PR CLT LEGMAR S,...,CAMISETA D2M MOTION CRERED/WHITE S,TENIS ULTRABOOST 21 LEP M CBLACK/CARBON/ACTTEA 42,TENIS X9000 BOOST L3 M CBLACK/CBLACK/GRETHR 40,CALCA NINI GRAFICA BLACK XL,TENIS BUSENITZ DRKBUR/FTWWHT/CROYAL 39,MOCHILA POWER V S VIVRED/WHITE/VISGRE NT,TENIS ULTRABOOST 5 0 DNA MORPHOLON W WONWHI/WONWHI/FOCOLI 38,VESTIDO FARM Q1 BOLRED/WHITE S,TENIS ULTRABOOST DNA CBLACK/CBLACK/CBLACK 38,JAQUETA OTR 1 2 ZIP W GRESIX XS
0,CAMISETA FL30 X GF GRA M BLACK/GRESIX XL,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.133631,...,0.133631,0.000000,0.000000,0.288675,0.000000,0.133631,0.000000,0.0,0.000000,0.144338
1,TENIS ZX 5K BOOST W HAZROS/AMBLUS/SONINK 35,0.0,1.000000,0.125988,0.235702,0.117851,0.000000,0.166667,0.000000,0.000000,...,0.000000,0.111111,0.201008,0.000000,0.125988,0.000000,0.100504,0.0,0.089087,0.000000
2,TENIS SEELEY FTWWHT/ENEBLU/ENERGY 43,0.0,0.125988,1.000000,0.133631,0.400892,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.125988,0.113961,0.000000,0.428571,0.000000,0.113961,0.0,0.101015,0.000000
3,TENIS MARATHON 5923 BLUE/SILVMT/CROYAL 41,0.0,0.235702,0.133631,1.000000,0.125000,0.111803,0.176777,0.000000,0.000000,...,0.000000,0.117851,0.106600,0.000000,0.267261,0.000000,0.106600,0.0,0.094491,0.000000
4,TENIS 3MC VULC CBLACK/FTWWHT/GLOPNK 34,0.0,0.117851,0.400892,0.125000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.235702,0.319801,0.000000,0.400892,0.000000,0.106600,0.0,0.377964,0.000000


In [19]:
input_descricao = "CAMISETA FL30 X GF GRA M BLACK/GRESIX XL"
recommendations = pd.DataFrame(df_t3.nlargest(11,input_descricao)['descricao'])
recommendations = recommendations[recommendations['descricao']!=input_descricao]
print(recommendations)

                                       descricao
417      CAMISETA FL30 X GF GRA M BLACK/GRESIX S
8358     CAMISETA FL30 X GF GRA M BLACK/LEGINK S
6501   CAMISETA FL30 X GF GRA M ACTMAR/ACTMAR XL
9448    CAMISETA FL30 X GF GRA M REABLU/CROYAL S
14714   CAMISETA FL30 X GF GRA M ACTORA/CBURGU M
5352                CAMISETA YOGA GRESIX/BLACK M
8551         CAMISETA TRAINING 365 LOGO BLACK XL
862               CAMISETA BASE PLAIN M BLACK XL
1851                CAMISETA B TEE LOGO BLACK XL
2557                CAMISETA TF POWER M BLACK XL


## Teste de qualidade

In [20]:
sample_test = df_t3.sample(20)

In [21]:
sample_test

descricao,descricao,CAMISETA FL30 X GF GRA M BLACK/GRESIX XL,TENIS ZX 5K BOOST W HAZROS/AMBLUS/SONINK 35,TENIS SEELEY FTWWHT/ENEBLU/ENERGY 43,TENIS MARATHON 5923 BLUE/SILVMT/CROYAL 41,TENIS 3MC VULC CBLACK/FTWWHT/GLOPNK 34,CONJUNTO W TS HD TGHT LEGINK/LEGINK/BLUE S,BERMUDA WAFFLE WONWHI L,MALA FAVOURITES BLACK/BLACK/WHITE NT,CAMISETA FLSPR A PR CLT LEGMAR S,...,CAMISETA D2M MOTION CRERED/WHITE S,TENIS ULTRABOOST 21 LEP M CBLACK/CARBON/ACTTEA 42,TENIS X9000 BOOST L3 M CBLACK/CBLACK/GRETHR 40,CALCA NINI GRAFICA BLACK XL,TENIS BUSENITZ DRKBUR/FTWWHT/CROYAL 39,MOCHILA POWER V S VIVRED/WHITE/VISGRE NT,TENIS ULTRABOOST 5 0 DNA MORPHOLON W WONWHI/WONWHI/FOCOLI 38,VESTIDO FARM Q1 BOLRED/WHITE S,TENIS ULTRABOOST DNA CBLACK/CBLACK/CBLACK 38,JAQUETA OTR 1 2 ZIP W GRESIX XS
10936,TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36,0.000000,0.235702,0.267261,0.250000,0.250000,0.0,0.176777,0.000000,0.000000,...,0.000000,0.117851,0.106600,0.000000,0.267261,0.000000,0.106600,0.000000,0.094491,0.000000
11685,TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43,0.000000,0.111111,0.503953,0.117851,0.353553,0.0,0.000000,0.000000,0.000000,...,0.000000,0.222222,0.201008,0.000000,0.377964,0.000000,0.201008,0.000000,0.089087,0.136083
12565,JAQUETA IVP VELOUR CHEWOO 2XS,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.109109,0.166667
10820,TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36,0.000000,0.222222,0.377964,0.235702,0.353553,0.0,0.166667,0.000000,0.000000,...,0.000000,0.111111,0.100504,0.000000,0.377964,0.000000,0.100504,0.000000,0.089087,0.000000
4377,TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38,0.000000,0.201008,0.113961,0.213201,0.106600,0.0,0.150756,0.000000,0.000000,...,0.000000,0.100504,0.090909,0.000000,0.113961,0.000000,0.181818,0.000000,0.161165,0.000000
3393,CALCA ADI CHINO BLACK 3434,0.144338,0.000000,0.154303,0.000000,0.144338,0.0,0.000000,0.272166,0.000000,...,0.000000,0.000000,0.000000,0.333333,0.154303,0.000000,0.000000,0.000000,0.000000,0.000000
5296,CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FO...,0.000000,0.000000,0.109109,0.000000,0.204124,0.0,0.000000,0.000000,0.000000,...,0.000000,0.096225,0.174078,0.000000,0.109109,0.000000,0.000000,0.000000,0.231455,0.000000
3184,SHORTS VIAGEM SAO PAULO TEAMAR XL,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.144338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6698,CAMISETA OFICIAL EVENTO WC22 ACTMAR M,0.133631,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,...,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6582,TENIS XPLR CBROWN/FTWWHT/TRACAR 40,0.000000,0.251976,0.285714,0.267261,0.267261,0.0,0.188982,0.000000,0.000000,...,0.000000,0.125988,0.227921,0.000000,0.285714,0.000000,0.113961,0.000000,0.101015,0.000000


In [22]:
lista_produtos_teste = []
for index, row in sample_test.iterrows():
    lista_produtos_teste.append(row['descricao'])
    print(row['descricao'])

TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36
TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43
JAQUETA IVP VELOUR CHEWOO 2XS
TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36
TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38
CALCA ADI CHINO BLACK 3434
CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FOOBLU 28
SHORTS VIAGEM SAO PAULO TEAMAR XL
CAMISETA OFICIAL EVENTO WC22 ACTMAR M
TENIS XPLR CBROWN/FTWWHT/TRACAR 40
TENIS SUPERSTAR W GREFIV/GREFIV/GRETHR 34
SHORT RESPONSE M BLACK/BLACK S 7"
TENIS DEERUPT RUNNER PARLEY W FTWWHT/FTWWHT/BLUSPI 34
TENIS ULTRABOOST OG W CBLACK/GRESIX/NGTFLA 38
CAMISETA 4 0 LOG BLACK/WHITE XL
CALCA TIGHT 3 STRIPES DUSPNK L
SHORTS LONG BLACK L
BONE ADIDAS DE REDINHA BLACK/BLACK/WHITE OSFM
CAMISETA 3STRIPES LS WHITE/BLACK L
TENIS FORUM LOW FTWWHT/FTWWHT/CBLACK 37


In [23]:
print(lista_produtos_teste)

['TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36', 'TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43', 'JAQUETA IVP VELOUR CHEWOO 2XS', 'TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36', 'TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38', 'CALCA ADI CHINO BLACK 3434', 'CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FOOBLU 28', 'SHORTS VIAGEM SAO PAULO TEAMAR XL', 'CAMISETA OFICIAL EVENTO WC22 ACTMAR M', 'TENIS XPLR CBROWN/FTWWHT/TRACAR 40', 'TENIS SUPERSTAR W GREFIV/GREFIV/GRETHR 34', 'SHORT RESPONSE M BLACK/BLACK S 7"', 'TENIS DEERUPT RUNNER PARLEY W FTWWHT/FTWWHT/BLUSPI 34', 'TENIS ULTRABOOST OG W CBLACK/GRESIX/NGTFLA 38', 'CAMISETA 4 0 LOG BLACK/WHITE XL', 'CALCA TIGHT 3 STRIPES DUSPNK L', 'SHORTS LONG BLACK L', 'BONE ADIDAS DE REDINHA BLACK/BLACK/WHITE OSFM', 'CAMISETA 3STRIPES LS WHITE/BLACK L', 'TENIS FORUM LOW FTWWHT/FTWWHT/CBLACK 37']


In [41]:
def recomenda_produto(produto):
    recommendations = pd.DataFrame(df_t3.nlargest(11,produto)['descricao'])
    recommendations = recommendations[recommendations['descricao']!=produto]
    return recommendations
    print(recommendations)
    
prod = "CALCA ADI CHINO BLACK 3434"
rec_prod = recomenda_produto(prod)
print(rec_prod)

                                  descricao
8034        CALCA CHINO AEROTECH BLACK 3032
12255          CALCA CHINO PANTS BLACK 3032
10688   CALCA CHINO STRIPED HEMP/WHITE 3434
8043       CALCA WORKSHOPPNTS BLACK/WHITE M
901       CALCA WIND STANDARD BLACK/WHITE L
9202      CALCA WIND STANDARD BLACK/WHITE M
13934          CALCA CHINO NORA LPURPL 3232
38      CALCA CHINO STRIPED HEMP/WHITE 3034
480            CALCA M V M BLACK/BLACK XL/L
2519   CALCA CLASSIC BLCKBRD BLACK/WHITE XL


In [28]:
type(rec_prod)

pandas.core.frame.DataFrame

In [45]:
print("PRODUTO BASE: CALCA ADI CHINO BLACK 3434")
rec_prod["descricao"].to_list()[7:]

PRODUTO BASE: CALCA ADI CHINO BLACK 3434


['CALCA CHINO STRIPED HEMP/WHITE 3034',
 'CALCA M V M BLACK/BLACK XL/L',
 'CALCA CLASSIC BLCKBRD BLACK/WHITE XL']

In [48]:
produtos_sample = ['TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36', 'TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43', 'JAQUETA IVP VELOUR CHEWOO 2XS', 'TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36', 'TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38', 'CALCA ADI CHINO BLACK 3434', 'CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FOOBLU 28', 'SHORTS VIAGEM SAO PAULO TEAMAR XL', 'CAMISETA OFICIAL EVENTO WC22 ACTMAR M', 'TENIS XPLR CBROWN/FTWWHT/TRACAR 40', 'TENIS SUPERSTAR W GREFIV/GREFIV/GRETHR 34', 'SHORT RESPONSE M BLACK/BLACK S 7"', 'TENIS DEERUPT RUNNER PARLEY W FTWWHT/FTWWHT/BLUSPI 34', 'TENIS ULTRABOOST OG W CBLACK/GRESIX/NGTFLA 38', 'CAMISETA 4 0 LOG BLACK/WHITE XL', 'CALCA TIGHT 3 STRIPES DUSPNK L', 'SHORTS LONG BLACK L', 'BONE ADIDAS DE REDINHA BLACK/BLACK/WHITE OSFM', 'CAMISETA 3STRIPES LS WHITE/BLACK L', 'TENIS FORUM LOW FTWWHT/FTWWHT/CBLACK 37']

produtos_desc = []
contador = 1
for i in produtos_sample:
    print(contador)
    print(i)
    print("...")
    for j in recomenda_produto(i)["descricao"].to_list()[7:]:
        produtos_desc.append(j)
        print(j)
    print("----------")
    contador+=1

1
TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36
...
TENIS STAN SMITH W FTWWHT/BLURUS/MAGOLD 36
TENIS STAN SMITH W FTWWHT/HALMIN/SUPCOL 34
TENIS STAN SMITH PRIDE FTWWHT/SHOPNK/RAYBLU 36
----------
2
TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43
...
TENIS ULTRA BOOST M FTWWHT/FTWWHT/FTWWHT 39
TENIS NITE JOGGER FTWWHT/FTWWHT/FTWWHT 43
TENIS TOP TEN FTWWHT/FTWWHT/FTWWHT 43
----------
3
JAQUETA IVP VELOUR CHEWOO 2XS
...
JAQUETA IVP DENIM DRKDEN S
CALCA IVP KNIT CHEWOO L
JAQUETA IVP ROBE SESOSL M
----------
4
TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36
...
TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 37
TENIS SUPERSTAR MULE FTWWHT/CBLACK/FTWWHT 36
TENIS SUPERSTAR C FTWWHT/FTWWHT/FTWWHT 31
----------
5
TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38
...
TENIS NMD R1 PK TRACAR/CBLACK/HIREGR 40
TENIS NMD R1 SPECTOO NGTCAR/ORGTIN/FTWWHT 41
TENIS NMD R1 PK TRACAR/CBLACK/HIREGR 43
----------
6
CALCA ADI CHINO BLACK 3434
...
CALCA CHINO STRIPED HEMP/WHITE 3034
CALCA M V M BLACK/BLACK XL/L
CALCA CLASSIC BLCKBRD BLACK/WHITE XL
--

In [49]:
print(produtos_desc)

['TENIS STAN SMITH W FTWWHT/BLURUS/MAGOLD 36', 'TENIS STAN SMITH W FTWWHT/HALMIN/SUPCOL 34', 'TENIS STAN SMITH PRIDE FTWWHT/SHOPNK/RAYBLU 36', 'TENIS ULTRA BOOST M FTWWHT/FTWWHT/FTWWHT 39', 'TENIS NITE JOGGER FTWWHT/FTWWHT/FTWWHT 43', 'TENIS TOP TEN FTWWHT/FTWWHT/FTWWHT 43', 'JAQUETA IVP DENIM DRKDEN S', 'CALCA IVP KNIT CHEWOO L', 'JAQUETA IVP ROBE SESOSL M', 'TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 37', 'TENIS SUPERSTAR MULE FTWWHT/CBLACK/FTWWHT 36', 'TENIS SUPERSTAR C FTWWHT/FTWWHT/FTWWHT 31', 'TENIS NMD R1 PK TRACAR/CBLACK/HIREGR 40', 'TENIS NMD R1 SPECTOO NGTCAR/ORGTIN/FTWWHT 41', 'TENIS NMD R1 PK TRACAR/CBLACK/HIREGR 43', 'CALCA CHINO STRIPED HEMP/WHITE 3034', 'CALCA M V M BLACK/BLACK XL/L', 'CALCA CLASSIC BLCKBRD BLACK/WHITE XL', 'CHUTEIRA NMZ MESSI TAN 18 4 IN FTWWHT/CBLACK/FOOBLU 39', 'CHUTEIRA NMZ MESSI TAN 18 4 TF FTWWHT/CBLACK/FOOBLU 41', 'CHUTEIRA NMZ MESSI TAN 18 3 TF FTWWHT/CBLACK/FOOBLU 37', 'JAQUETA APRESENTACAO SAO PAULO TEAMAR L', 'JAQUETA HINO SAO PAULO TACRED XL', 'S

In [51]:
print(len(produtos_desc))

60


In [53]:
df_t2.head(3)

,carrinho,produto,descricao,categoria
0,1439746,4060511132955,CAMISETA FL30 X GF GRA M BLACK/GRESIX XL,TRAINING
1,4238889,4064055964928,TENIS ZX 5K BOOST W HAZROS/AMBLUS/SONINK 35,ORIGINALS
2,229032,4057283660168,TENIS SEELEY FTWWHT/ENEBLU/ENERGY 43,SKATEBOARDING


In [65]:
def id_by_desc(desc):
    return df_t2.loc[df_t2['descricao'] == desc][:1]["produto"].iloc[0]

In [66]:
for i in produtos_desc:
    print(id_by_desc(i))
    print(i)
    print("---")

4065419836103
TENIS STAN SMITH W FTWWHT/BLURUS/MAGOLD 36
---
4064047250466
TENIS STAN SMITH W FTWWHT/HALMIN/SUPCOL 34
---
4064041776504
TENIS STAN SMITH PRIDE FTWWHT/SHOPNK/RAYBLU 36
---
4056567457142
TENIS ULTRA BOOST M FTWWHT/FTWWHT/FTWWHT 39
---
4051043513466
TENIS NITE JOGGER FTWWHT/FTWWHT/FTWWHT 43
---
4055341488990
TENIS TOP TEN FTWWHT/FTWWHT/FTWWHT 43
---
4065424773295
JAQUETA IVP DENIM DRKDEN S
---
4065431537316
CALCA IVP KNIT CHEWOO L
---
4065427175539
JAQUETA IVP ROBE SESOSL M
---
4064047226966
TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 37
---
4060519363757
TENIS SUPERSTAR MULE FTWWHT/CBLACK/FTWWHT 36
---
4059808636078
TENIS SUPERSTAR C FTWWHT/FTWWHT/FTWWHT 31
---
4059319445831
TENIS NMD R1 PK TRACAR/CBLACK/HIREGR 40
---
4064053861373
TENIS NMD R1 SPECTOO NGTCAR/ORGTIN/FTWWHT 41
---
4059319449914
TENIS NMD R1 PK TRACAR/CBLACK/HIREGR 43
---
4060512737661
CALCA CHINO STRIPED HEMP/WHITE 3034
---
4062051493374
CALCA M V M BLACK/BLACK XL/L
---
4058031570548
CALCA CLASSIC BLCKBRD BLACK